In [13]:
import numpy as np

from human_terms_network import Human_Terms_Network
from dataset_load import *
from lr_baseline import LRBaseline

In [2]:
# open_pickle in dataset_load

X_train_original, X_test_original, y_train_original, y_test_original = load_imdb('../../data/imdb')

In [3]:
word_list, connotation = load_unigrams('./imdb-unigrams.txt', X_train_original, y_train_original)

In [4]:
y_train_agreement, y_test_agreement = generate_appearance(X_train_original, X_test_original, 
                                                          word_list, connotation)

In [5]:
imdb_baseline = LRBaseline(X_train_original, X_test_original, y_train_original, y_test_original, 
                           human_terms=word_list)

In [6]:
print(imdb_baseline.baseline())

(0.94412, 0.85576)


In [7]:
X_train = imdb_baseline.baseline_cv.transform(X_train_original)
X_test = imdb_baseline.baseline_cv.transform(X_test_original)
y_train = y_train_original
y_test = y_test_original

In [14]:
imdb_htm = Human_Terms_Network(input_shape=X_train.shape[1],
                               human_terms_shape=len(word_list),
                              domain='imdb')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 3641)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 3642      
Total params: 3,642
Trainable params: 3,642
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 83)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 3641)         0                                            
_________________________________________

In [15]:
imdb_htm.combined.load_weights('./combined/combined-weight-improvement-10-0.34.hdf5')

In [16]:
imdb_htm.set_data(X_train, X_test, y_train_agreement, y_test_agreement, y_train, y_test)

In [32]:
imdb_htm.test(reject=False)

25000/25000 [==============================] - 1s 56us/step


ValueError: too many values to unpack (expected 2)

In [30]:
from keras.models import Model

human_terms_relu_model = Model(inputs=imdb_htm.combined.input,
                                            outputs=imdb_htm.combined.get_layer('concatenate').output)
predict_relu = human_terms_relu_model.predict([X_test, y_test_agreement])
accept_indices = np.where(np.sum(predict_relu, axis=1)!=0)
total_reject = X_test.shape[0] - len(accept_indices)
rejection_rate = total_reject/X_test.shape[0]

In [23]:
rejection_rate

0.99996

(25000, 83)

In [26]:
eval = imdb_htm.combined.evaluate([imdb_htm.X_test, imdb_htm.y_test_agreement], imdb_htm.y_test)

25000/25000 [==============================] - 1s 54us/step


In [20]:
eval

[0.7428042534065247, 0.7199926891326904, 0.33156]